In [2]:
!pip install transformers torch-geometric tqdm
!pip install Pillow==9.4.0
!pip install fastai==2.7.14
# !pip install fastbook==0.0.29

In [3]:
!pip install wandb

In [4]:
import os
import sys
import wandb
import random
import time
import argparse
import torch
import pickle
import numpy as np
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.utils import clip_grad_norm_

import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import get_constant_schedule, get_constant_schedule_with_warmup

from sklearn.metrics import accuracy_score, f1_score, classification_report
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [5]:
wandb.login()

wandb: Currently logged in as: raj8lm (mh_detection). Use `wandb login --relogin` to force relogin


True

In [6]:
os.chdir('/content/gdrive/My Drive/MTech_Thesis/SKAIG_ERC/SKAIG-ERC')

In [7]:
sys.path.insert(0,'/content/gdrive/My Drive/MTech_Thesis/SKAIG_ERC/SKAIG-ERC')

In [8]:

import os, re, csv
import pandas as pd
from tqdm import tqdm

import importlib
# importlib.reload(importlib.import_module('gnn_dataset'))

from gnn_dataset import BaseDataset, collate_fn, collate_fn_batch
# from gnn import *
from gnn import MentalModel, BatchMentalModel, MentalModelWithSpectrogram, EmotionDetectionUsingSpectrogram

# from fastai.vision.models import resnet34
# from fastai.vision.all import *
# from fastai.vision.all import PILImage, tensor

from gnn_for_meld_emorynlp import BatchMentalModelResidual
from fastai.vision.all import PILImage, tensor

import warnings
warnings.filterwarnings('ignore')

IEMOCAP
The Data below is the Test Data that we tried to print here. The first item is the "utt", which is "UTTERANCES" in short. This is the words that is actually spoken in the sentence.

The next section is the label for the sentence. Each label identifies a sentiment:     label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}

The next section is the "spk", i.e., the SPEAKERS that are part of the conversation - like M for male and F for Female. This is a conversation between 2 people.

The final section is the "graph", where the graph's edge indices and the edge values are mentioned.

Eg for understanding:

```
'Why does that bother you?',  2,  M
'Well, maybe maybe he just wanted to see her again.',   2,  M
'Nobody comes seven hundred miles just to see.',  2,  F
"What are you talking about?  He grew up next to the girl his whole life, why wouldn't he want to see her again?  Don't look at me like that.  He hasn't told me anything he didn't told you.",   5,  M
"Why do you think he's even thinking that?",  2,  M
"He's got that about it.",  2,  F
'Well, so what?',  5,   M
"What's going on here, Joe?",   3,  F
'Now listen.',  5,  M
"How do you know why she's waited?",  5,  M
"Look, it's a beautiful day outside, why are we arguing?",  5,  M
'Well, what do you want me to do about it?  What do you want?',   5,  M
"I want you to pretend like he's coming back.  Both of you, don't think I haven't noticed you and Chris since she's gotten here.  I don't want any nonsense.",  5,  F
'But Kate?',  5,  M
"No, because if she- he's not coming back then I'll kill myself. Laugh!  Laugh at me!",   5,  F
'Calm yourself.',   5,  M
'Calm yourself.',   5,  M
'Only last week a man turned up in Detroit missing longer than Larry.  You read it yourself.',  5,  F
'All right.  All right.',   5,  M
'You above all have got to believe.',   3,  F
"What's that suppose to mean, me above all?  Look at you, you're shaking.",   3,  M
"I can't help it.",   3,  F
'What have I got to hide, Kate?  What the hell is the matter with you?',   3,   M
```


Labels
[2, 2, 2, 5, 2, 2, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3]

Speakers
['M', 'M', 'F', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'M', 'M', 'F', 'M', 'F', 'M', 'M', 'F', 'M', 'F', 'M', 'F', 'M']

### OPTIONAL: BREAKING THE DATA TO UNDERSTAND THE STRUCTURE OF PICKER FILE

In [ ]:
def show_data(dataset_name, hip):
    print(dataset_name)
    dataset_path = 'bert_data/' + dataset_name + '/' + dataset_name + '_graph_hip' + str(hip) + '_new.pkl'
    data_utf = pickle.load(open(dataset_path, 'rb'), encoding='utf-8')

    # users should use these instructions to load pkl dataset. \
    # videoIDs, videoSpeakers, videoLabels, videoText,\
    # videoAudio, videoVisual, videoSentence, trainVid,\
    # testVid = pickle.load(open(dataset_path, 'rb'), encoding='latin1')

    # data = data_utf['test']

    '''
        label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
    '''
    # utt = data[0]
    # label = data[1]
    # spk = data[2]
    # graph = data[3]

    # print(f"UTTERANCE: {np.matrix(utt)}")
    # print(f"LABEL: {np.matrix(label)}")
    # print(f"SPEAKER GENDER: {np.matrix(spk)}")
    # print(f"GRAPH DATA: {np.matrix(graph)}")

    # Loading the pickle file to a dataframe
    unpickled_df = pd.read_pickle(dataset_path)

    df = pd.DataFrame(unpickled_df.items())
    # Getting the Train Data
    train_val = df[[1]].iloc[0]
    val_arr = train_val.to_numpy()

    train_df=pd.DataFrame()
    train_df["UTTERANCE"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][0]])
    train_df["LABEL"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][1]])
    train_df["SPEAKER GENDER"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][2]])
    # Graph Data is again a Dict
    train_df["GRAPH DATA_EDGE_INDEX"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][3]['edge_index']])
    train_df["GRAPH DATA_EDGE_TYPE"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][3]['edge_type']])
    print(train_df.shape)

    test_val = df[[1]].iloc[1]
    val_arr = test_val.to_numpy()

    test_df=pd.DataFrame()
    test_df["UTTERANCE"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][0]])
    test_df["LABEL"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][1]])
    test_df["SPEAKER GENDER"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][2]])
    # Graph Data is again a Dict
    test_df["GRAPH DATA_EDGE_INDEX"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][3]['edge_index']])
    test_df["GRAPH DATA_EDGE_TYPE"] = pd.DataFrame([' '.join(str(j) for j in i) for i in val_arr[0][3]['edge_type']])
    print(test_df.shape)

    return train_df, test_df


train_df, test_df = show_data("IEMOCAP", 7)
train_df.head(10)

IEMOCAP
(120, 5)
(31, 5)


,UTTERANCE,LABEL,SPEAKER GENDER,GRAPH DATA_EDGE_INDEX,GRAPH DATA_EDGE_TYPE
0,"Hey, Isabella. Hey, how's it going, Joe? I'm a...",2 0 2 0 0 2 2 0 0 0 4 4 4 4 4 4 2 0 4 0 4 4 4 ...,M F M F F M M F F F M F F M F F M F M F M F M ...,[ 0 0 0 ... 55 55 55] [ 0 1 2 ... 51 53 55],xEffect oWant xWant oWant oWant xWant xWant oW...
1,"Oh, you're never going to believe it. What? Wh...",4 4 4 4 4 4 4 4 4 4 4 4 0 0 0 4 0 4 4 0 0 4 4 ...,M F M F F F M F F F M M F M M F M F F M M M F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant oWant oWant oWant xWant oW...
2,"Oh, man, I just -- I don't know what to do. I...",5 5 2 5 5 2 2 2 5 2 2 5 5 5 2 5 2 5 2 5 5 2 5 ...,M M F M M F F M M F F M M M F M F M F M M F M ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect xWant oWant xWant xWant oWant oWant xW...
3,What time is it? They're supposed to run arou...,0 2 4 5 5 5 5 5 4 2 0 3 5 3 5 5 2 5 5 5 3 5 4 ...,M F M M M F M M M M M F M F M F M F M F F M M ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant xWant xWant oWant xWant xW...
4,"We'll be alright, darling. Whatever happens, ...",2 4 4 4 4 4 4 4 4 2 2 4 3 2 4 4 1 4 4 4 4 4 4 ...,M M F M F M F F F M M F M F F F M F F M F F F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect xWant oWant xWant oWant xWant oWant oW...
5,I'm just so tired all the time. Well have you ...,1 2 5 5 5 2 2 3 5 5 1 1 3 5 4 5 5 5 5 2 2 2 5 ...,M F M F M F F M M M F F M M F M M M M M F F M ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant oWant xWant oWant oWant xW...
6,Hello. Thank you for calling Dell Taco Corpor...,0 5 2 5 2 5 2 5 5 2 2 5 5 2 5 2 2 2 3 3 3 5 3 ...,F M M M F M F M F F F M M F F F M F M M M F M ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant oWant oWant xWant oWant xWant oW...
7,"Kathy died last night. yeah, I mean, you know,...",1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ...,M M M F M F M F M M F M F M F M F M F M M F M ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect xWant xWant oWant xWant oWant xWant oW...
8,"As a matter of fact, the real cause of that ro...",2 4 3 4 2 3 2 3 2 2 3 2 5 2 2 2 2 2 2 3 5 2 2 ...,M F M F F M F M F F M F F F M M F M F F M F M ...,[ 0 0 0 ... 56 56 56] [ 0 1 2 ... 50 52 56],xEffect oWant xWant oWant oWant xWant oWant xW...
9,Hello? Oh God I finally got...you know how lon...,2 5 2 5 2 5 2 5 2 3 3 5 2 5 2 5 2 5 2 2 5 2 5 ...,M F M F M F M F M F F F M F M F M F M M F M F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant oWant xWant oWant xWant oW...


In [ ]:
test_df.head(10)

,UTTERANCE,LABEL,SPEAKER GENDER,GRAPH DATA_EDGE_INDEX,GRAPH DATA_EDGE_TYPE
0,"Why does that bother you? Well, maybe maybe he...",2 2 2 5 2 2 5 3 5 5 5 5 5 5 5 5 5 5 5 3 3 3 3,M M F M M F M F M M M M F M F M M F M F M F M,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect xWant oWant xWant xWant oWant xWant oW...
1,What is it? um. I'm sorry. It's just a lot ah...,2 1 1 1 1 3 3 5 1 5 1 1 3 1 1 5 3 5 3 5 5 5 5 ...,M F F F F M M M F M F F M F F F M F M M F F F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant oWant oWant oWant xWant xWant xW...
2,Hi. Um- My luggage didn't come out of the con...,2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 5 2 ...,M F M F M F M F M F M F M F M F M F M F F M F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant oWant xWant oWant xWant oW...
3,"Can I help you? Oh, this- this form that you h...",2 2 2 2 2 5 5 3 2 5 5 5 5 2 5 5 5 2 5 5 5 5 5 ...,F F F F F M F M F M F M F F F M F F M F M F F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect xWant xWant xWant xWant oWant xWant oW...
4,"Hi,um- I think my baggage was lost, and I need...",2 2 2 2 2 2 5 2 2 2 2 2 2 2 2 2 2 5 2 5 2 2 5 ...,F M F M F M F M F F M M F M M F M F M F M F M ...,[ 0 0 0 ... 90 90 90] [ 0 1 2 ... 86 88 90],xEffect oWant xWant oWant xWant oWant xWant oW...
5,What time is it? They're supposed to run arou...,4 1 4 4 2 4 4 4 4 4 4 4 5 4 4 5 4 4 4 5 5 5 4 ...,M F M M F M M M M M M M M M M F M M M F M F M ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant xWant oWant xWant xWant xW...
6,You're the only one I know who loves his paren...,0 0 0 0 0 2 1 0 2 2 0 0 4 4 4 0 4 0 0 1 1 1 1 ...,F M F F F M F M M F F M M M M F M F F M F M M ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant xWant xWant oWant xWant oW...
7,What's he going to say? Maybe we should tell h...,1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 5 5 3 5 5 5 ...,F M M M M F M F M F F M F M M M M M M M F M M ...,[ 0 0 0 ... 52 52 52] [ 0 1 2 ... 50 51 52],xEffect oWant oWant oWant oWant xWant oWant xW...
8,"Hi, I need an ID. ahh Yeah, this is the wrong ...",2 2 5 5 5 2 5 5 5 5 5 5 5 5 2 5 5 5 5 5 5 5 5 ...,F M F F F M F M F M F M F M F M F M F M F M F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant xWant xWant oWant xWant oW...
9,Why does that bother you? She's been in New Yo...,2 2 2 2 5 5 5 5 3 5 5 5 5 5 5 3 5 3 5 3 5 5 3 ...,M F M M M M F M F F M F M F M F M F M F M M F ...,[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,xEffect oWant xWant xWant xWant xWant oWant xW...


## DATA-LOADING

In [9]:
def get_train_valid_sampler(trainset, valid=0.1):
    size = len(trainset)
    idx = list(range(size))
    split = int(valid * size)
    return SubsetRandomSampler(idx[split:]), SubsetRandomSampler(idx[:split])

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

def get_loaders(dataset_name, hip, batch_size=8, pretrained_model='bert-base-uncased', valid=0.1, shuffle=True):

    tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
    trainset = BaseDataset(dataset_name, hip, 'train', tokenizer)
    testset = BaseDataset(dataset_name, hip, 'test', tokenizer)
    # Optional printing of dataset_name's test part to understand the data. \
    # The understanding of IEMOCAP is mentioned in a comment above.
    # show_data(dataset_name, hip)

    if dataset_name == 'IEMOCAP':
        train_sampler, valid_sampler = get_train_valid_sampler(trainset, valid)
        train_loader = DataLoader(trainset,
                                  batch_size=8,
                                  sampler=train_sampler,
                                  num_workers=0,
                                  collate_fn=collate_fn,
                                  worker_init_fn=seed_worker)
        dev_loader = DataLoader(trainset,
                                batch_size=1,
                                sampler=valid_sampler,
                                num_workers=0,
                                collate_fn=collate_fn,
                                worker_init_fn=seed_worker)
        test_loader = DataLoader(testset,
                                 batch_size=1,
                                 shuffle=shuffle,
                                 num_workers=0,
                                 collate_fn=collate_fn,
                                 worker_init_fn=seed_worker)
    else:
        devset = BaseDataset(dataset_name, hip, 'dev', tokenizer)
        train_loader = DataLoader(trainset,
                                  batch_size=batch_size,
                                  shuffle=shuffle,
                                  num_workers=0,
                                  collate_fn=collate_fn_batch,
                                  worker_init_fn=seed_worker)
        dev_loader = DataLoader(devset,
                                batch_size=batch_size,
                                shuffle=shuffle,
                                num_workers=0,
                                collate_fn=collate_fn_batch,
                                worker_init_fn=seed_worker)
        test_loader = DataLoader(testset,
                                 batch_size=batch_size,
                                 shuffle=shuffle,
                                 num_workers=0,
                                 collate_fn=collate_fn_batch,
                                 worker_init_fn=seed_worker)
    return train_loader, dev_loader, test_loader

### STARTING WITH TRAINING - WITHOUT SPECTROGRAM

In [ ]:
def train(dataset_name, model, loss_func, trainloader, devloader, testloader,
          n_epochs, optimizer, scheduler, training_step, model_path, log_path,
          metric_path, use_gpu, window, mode):

    model = model.to('cuda:0')
    model.train()

    f = open(log_path, 'a+', encoding='utf-8')

    train_loss_list = []
    train_f1_list = []
    dev_f1_list = []
    test_f1_list = []

    best_fscore = 0
    best_mf1 = 0
    best_accuracy = 0
    best_report = None

    best_test_fscore = 0
    best_test_mf1 = 0
    best_test_accuracy = 0
    best_test_report = None

    step = 0
    early_stopping_step = 0

    for epoch in range(n_epochs):
        losses = []
        preds = []
        labels = []
        masks = []
        num_utt = 0
        print('Epoch {} start: '.format(epoch + 1))
        if step > training_step:
            break
        start_time = time.time()
        for data in trainloader:
            # (clen, slen), (clen)
            # Each data that is coming our way - But the entire data is encoded here.
            textf, wrdm, label, uttm, spkm, edge_index, edge_attr, _, _, conv = data
            if dataset_name == 'IEMOCAP':
                if use_gpu:
                    textf = textf.cuda()
                    wrdm = wrdm.cuda()
                    uttm = uttm.cuda()
                    spkm = spkm.cuda()
                    edge_index = edge_index.cuda()
                    edge_attr = edge_attr.cuda()
                logits = model(textf, wrdm, uttm, spkm, window, mode, edge_index, edge_attr, residual=False)
            else:
                conv_len = [int(torch.sum(um).item()) for um in uttm]  # torch.sum(uttm, dim=1).numpy().tolist()
                if dataset_name == 'DailyDialog':
                    logits = model(textf, wrdm, conv_len, uttm, spkm, window, mode, edge_index, edge_attr, use_gpu, residual=False)
                else:
                    logits = model(textf, wrdm, conv_len, edge_index, edge_attr, use_gpu)
                label = torch.cat(label, dim=0)
            if use_gpu:
                label = label.cuda()
            # loss = loss_func(torch.log_softmax(logits, dim=-1), label)
            # print(f"LOGITS WITHOUT SPECTROGRAM: {logits} | LABELS: {label}")
            loss = loss_func(logits, label)

            model.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            pred_ = torch.argmax(torch.softmax(logits, dim=-1), dim=1)
            preds.append(pred_.cpu().numpy())
            labels.append(label.data.cpu().numpy())
            losses.append(loss.item() * label.size(0))
            num_utt += label.size(0)
            step += 1
            if step > training_step:
                break

        preds = np.concatenate(preds)
        labels = np.concatenate(labels)

        avg_loss = np.round(np.sum(losses) / num_utt, 4)
        avg_accuracy = round(accuracy_score(labels, preds) * 100, 5)
        if dataset_name == 'DailyDialog':
            avg_fscore = round(f1_score(labels, preds, labels=[0, 2, 3, 4, 5, 6], average='micro') * 100, 5)
        else:
            avg_fscore = round(f1_score(labels, preds, average='weighted') * 100, 5)
        train_mf1 = round(f1_score(labels, preds, average='macro') * 100, 5)
        dev_accuracy, dev_fscore, dev_mf, dev_reports = evaluate(dataset_name, model, devloader, use_gpu, window=window, mode=mode)
        test_accuracy, test_fscore, test_mf, test_reports = evaluate(dataset_name, model, testloader, use_gpu, window=window, mode=mode)

        print(f"DEV-ACCURACY: {dev_accuracy} || BEST=ACCURACY: {best_accuracy}")
        # Log the Avg. Train Accuracy
        wandb.log({'avg_train_accuracy':avg_accuracy})
        # Log the Train Macro F1 Score
        wandb.log({'train_macro_f1_score':train_mf1})
        # Log the Dev Accuracy
        wandb.log({'dev_accuracy':dev_accuracy})
        # Log the Test Accuracy
        wandb.log({'test_accuracy':test_accuracy})

        if dev_accuracy > best_accuracy:
        # if dev_fscore > best_fscore:
            best_fscore = dev_fscore
            best_mf1 = dev_mf
            best_accuracy = dev_accuracy
            best_report = dev_reports

            best_test_fscore = test_fscore
            best_test_mf1 = test_mf
            best_test_accuracy = test_accuracy
            best_test_report = test_reports

            early_stopping_step = 0
            print("Best Accuracy Till thie Epoch:", best_accuracy)
            # Saving the model
            torch.save(model.state_dict(), model_path)
        else:
            early_stopping_step += 1

        print(f"CURRENT VALUE OF STOPPING: {early_stopping_step}")
        train_loss_list.append(avg_loss)
        # Log the Avg. train loss
        wandb.log({'avg_train_loss':avg_loss})
        train_f1_list.append(avg_fscore)
        # Log the Avg. train f1 score
        wandb.log({'avg_train_fscore':avg_fscore})
        dev_f1_list.append(dev_fscore)
        # Log the Avg. dev f1 score
        wandb.log({'avg_dev_fscore':dev_fscore})
        test_f1_list.append(test_fscore)
        # Log the Avg. test f1 score
        wandb.log({'avg_test_fscore':test_fscore})

        log = 'Train: Epoch {} Loss {}, ACC {}, F1 {}, mF {}'.format(epoch + 1, avg_loss,
                                                                     avg_accuracy, avg_fscore, train_mf1)
        print(log)
        f.write(log + '\n')
        log = 'Validation: ACC {}, F1 {}, mF {}'.format(dev_accuracy, dev_fscore, dev_mf)
        print(log)
        f.write(log + '\n')
        log = 'Test: ACC {}, F1 {}, mF {}'.format(test_accuracy, test_fscore, test_mf)
        print(log)
        f.write(log + '\n')
        print('Epoch {} finished. Elapse {}'.format(epoch + 1, round(time.time() - start_time, 4)))
        if early_stopping_step == 50:
            break
    print('----------------------------------------------')
    f.write('----------------------------------------------')
    log = '\n\n[DEV] best ACC {}, F1 {}, mF {}'.format(best_accuracy, best_fscore, best_mf1)
    f.write(log + '\n')
    print(log)
    f.write(best_report)
    log = '[TEST] best ACC {}, F1 {}, mF {}'.format(best_test_accuracy, best_test_fscore, best_test_mf1)
    f.write(log + '\n')
    print(log)
    f.write(best_test_report)
    f.write('----------------------------------------------\n')
    f.close()
    dump_data = [train_loss_list, train_f1_list, dev_f1_list, test_f1_list]
    pickle.dump(dump_data, open(metric_path, 'wb'))


def evaluate(dataset_name, model, dataloader, use_gpu, window, mode):
    model.eval()
    preds = []
    labels = []

    with torch.no_grad():
        for data in dataloader:
            textf, wrdm, label, uttm, spkm, edge_index, edge_attr, _, _, conv  = data
            if dataset_name == 'IEMOCAP':
                if use_gpu:
                    textf = textf.cuda()
                    wrdm = wrdm.cuda()
                    uttm = uttm.cuda()
                    spkm = spkm.cuda()
                    edge_index = edge_index.cuda()
                    edge_attr = edge_attr.cuda()
                logits = model(textf, wrdm, uttm, spkm, window, mode, edge_index, edge_attr, residual=False)
            else:
                conv_len = [int(torch.sum(um).item()) for um in uttm]  # torch.sum(uttm, dim=1).numpy().tolist()
                if dataset_name == 'DailyDialog':
                    logits = model(textf, wrdm, conv_len, uttm, spkm, window, mode, edge_index, edge_attr, use_gpu, residual=False)
                else:
                    logits = model(textf, wrdm, conv_len, edge_index, edge_attr, use_gpu)
                # logits = model(textf, wrdm, conv_len, edge_index, edge_attr, use_gpu)
                label = torch.cat(label, dim=0)
            pred_ = torch.argmax(torch.softmax(logits, dim=1), dim=1)
            preds.append(pred_.cpu().numpy())
            labels.append(label.data.numpy())
    preds = np.concatenate(preds)
    labels = np.concatenate(labels)

    avg_accuracy = round(accuracy_score(labels, preds) * 100, 5)

    if dataset_name == 'DailyDialog':
        avg_fscore = round(f1_score(labels, preds, labels=[0, 2, 3, 4, 5, 6], average='micro') * 100, 5)
        report_classes = classification_report(labels, preds, labels=[0, 2, 3, 4, 5, 6], digits=4)
    else:
        avg_fscore = round(f1_score(labels, preds, average='weighted') * 100, 5)
        report_classes = classification_report(labels, preds, digits=4)
    mf1 = round(f1_score(labels, preds, average='macro') * 100, 5)

    model.train()

    return avg_accuracy, avg_fscore, mf1, report_classes


In [ ]:
beta=True
batch_size=20
bias=False
dataset_name='IEMOCAP'
# dataset_name='DailyDialog'
encoder_mode='maxpooling'
tr_nhead=6
tr_ff_dim=300
tr_dropout=0.1
attn_mask=True
tr_num_layer=6
max_len=120
bidirectional=True
num_block = 3
cn_nhead=6
cn_ff_dim=600
cn_dropout=0.1
edge_dim=300
cn_num_layer=5
edge_mapping=True
root_weight=True
# choice='cn'
choice='both'
valid=0.1
warmup_step=1000
schedule='linear'
training_step=10000
seed=7
n_epochs=150
lr=1e-5
l2=0
use_gpu=True
window=10
mode='uso'

if dataset_name == 'IEMOCAP':
    num_class = 6
else:
    num_class = 7

model_index = str(29)
model_dir = dataset_name + '_C/model'
log_dir = dataset_name + '_C/logs/log'
metric_dir = dataset_name + '_C/logs/metric'

model_path = model_dir + model_index + '.pth'
log_path = log_dir + model_index + '.txt'
metric_path = metric_dir + model_index + '.pkl'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if dataset_name == 'IEMOCAP':
  pretrain='roberta-base'
  sent_dim=300
  hip=7
  print(f"The model getting trained is=> DATASET: {dataset_name}, pretrain: {pretrain}")
  model = MentalModel(encoder_type=pretrain, encoder_mode=encoder_mode, sent_dim=sent_dim, tr_nhead=tr_nhead,
                      tr_ff_dim=tr_ff_dim, tr_dropout=tr_dropout, attn_mask=attn_mask, tr_num_layer=tr_num_layer,
                      max_len=max_len, num_class=num_class, bidirectional=bidirectional, num_block=num_block,
                      cn_nhead=cn_nhead, cn_ff_dim=cn_ff_dim, cn_dropout=cn_dropout, edge_dim=edge_dim, bias=bias,
                      cn_num_layer=cn_num_layer, edge_mapping=edge_mapping, beta=beta, root_weight=root_weight, choice=choice)

elif dataset_name == 'DailyDialog':
  pretrain='roberta-large'
  sent_dim=300
  hip=2
  print(f"The model getting trained is=> DATASET: {dataset_name}, pretrain: {pretrain}")
  model = BatchMentalModel(pretrain, encoder_mode, sent_dim, tr_nhead,
                      tr_ff_dim, tr_dropout, attn_mask, tr_num_layer,
                      max_len, num_class, bidirectional, num_block,
                      cn_nhead, cn_ff_dim, cn_dropout, edge_dim, bias,
                      cn_num_layer, edge_mapping, beta, root_weight, choice)
else:
  print(f"The model getting trained is=> DATASET: {dataset_name}, pretrain: {pretrain}")
  model = BatchMentalModelResidual(args.pretrain, args.encoder_mode, args.sent_dim,
                                    args.cn_ff_dim, args.cn_nhead, args.cn_dropout,
                                    args.edge_dim, num_class, bias, args.cn_num_layer,
                                    edge_mapping, beta, root_weight, args.residual_type)

The model getting trained is=> DATASET: IEMOCAP, pretrain: roberta-base


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch.optim as optim

train_loader, dev_loader, test_loader = get_loaders(dataset_name, hip, batch_size, pretrain, valid, True)
loss_func = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=l2)

if schedule == 'linear':
    scheduler = get_linear_schedule_with_warmup(optimizer, warmup_step, training_step)
elif schedule == 'warmup':
    scheduler = get_constant_schedule_with_warmup(optimizer, warmup_step)
else:
    scheduler = get_constant_schedule(optimizer)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

## TRAINING WITHOUT SPECTROGRAM

In [9]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="SKAIG",
    name="SKAIG-Without-Emotion-Detection-Using-Sound-With-both-encoding",
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.01,
    "architecture": "ROBERTA And Graph Neural",
    "dataset": "IEMOCAP",
    "epochs": 150,
    }
)

wandb: Currently logged in as: raj8lm (mh_detection). Use `wandb login --relogin` to force relogin


In [ ]:
# IEMOCAP
train(dataset_name, model, loss_func, train_loader, dev_loader, test_loader, n_epochs, optimizer, scheduler,
            training_step, model_path, log_path, metric_path, use_gpu, window=window, mode=mode)

# # DAILY DIALOG
# train(dataset_name, model, loss_func, train_loader, dev_loader, test_loader, n_epochs, optimizer, scheduler,
#             training_step, model_path, log_path, metric_path, use_gpu, window=window, mode=mode)

Epoch 1 start: 
DEV-ACCURACY: 24.58101 || BEST=ACCURACY: 0
Best Accuracy Till thie Epoch: 24.58101
CURRENT VALUE OF STOPPING: 0
Train: Epoch 1 Loss 1.854, ACC 26.39456, F1 11.04756, mF 6.97591
Validation: ACC 24.58101, F1 9.70013, mF 6.57698
Test: ACC 23.65989, F1 9.05371, mF 6.37768
Epoch 1 finished. Elapse 15.5971
Epoch 2 start: 
DEV-ACCURACY: 24.58101 || BEST=ACCURACY: 24.58101
CURRENT VALUE OF STOPPING: 1
Train: Epoch 2 Loss 1.9437, ACC 25.12019, F1 10.2487, mF 6.85523
Validation: ACC 24.58101, F1 9.70013, mF 6.57698
Test: ACC 23.65989, F1 9.05371, mF 6.37768
Epoch 2 finished. Elapse 13.8798
Epoch 3 start: 
DEV-ACCURACY: 24.58101 || BEST=ACCURACY: 24.58101
CURRENT VALUE OF STOPPING: 2
Train: Epoch 3 Loss 1.8743, ACC 26.65563, F1 11.79389, mF 7.40941
Validation: ACC 24.58101, F1 9.70013, mF 6.57698
Test: ACC 23.65989, F1 9.05371, mF 6.37768
Epoch 3 finished. Elapse 12.1956
Epoch 4 start: 
DEV-ACCURACY: 24.58101 || BEST=ACCURACY: 24.58101
CURRENT VALUE OF STOPPING: 3
Train: Epoch 4 L

#### IEMOCAP RESULTS
[DEV] best ACC 27.93296, F1 21.48058, mF 14.45441

[TEST] best ACC 25.93962, F1 19.25896, mF 14.36226

#### NEW IEMOCAP RESULTS AFTER 78 EPOCHS
[DEV] best ACC 61.08007, F1 58.82376, mF 55.18324

[TEST] best ACC 59.76587, F1 56.94629, mF 52.72917

In [ ]:
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

avg_dev_fscore,▁▁▂▂▂▁▂▂▂▂▂▅▄▅▅▆▆▅▇▇▇█▆▇█▇▇▇█▇▇█▇▆▇██▇▇▇
avg_test_fscore,▁▁▂▂▁▁▂▂▂▂▁▅▆▅▆▇▆▆▇▇▇▇▆██▇███▇█▇▇▇██████
avg_train_accuracy,▂▁▂▂▂▂▁▁▂▃▂▃▄▃▄▅▅▅▅▆▆▆▇▆▆▆▇▇▇▇▇▇▇▇▇▇████
avg_train_fscore,▁▁▁▁▁▁▂▁▂▂▂▂▃▄▅▅▅▅▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇███
avg_train_loss,███▇▇▇▆█▇▇▇▇▆▅▅▅▄▄▄▃▄▃▃▃▃▂▃▃▂▂▂▂▂▂▁▂▂▁▁▁
dev_accuracy,▁▁▁▁▁▁▂▁▂▂▂▄▅▄▄▅▅▆█▅▇▇▆▇▇▆▇▆▇▆██▆██▇▇███
test_accuracy,▁▁▁▁▁▁▂▁▁▂▃▄▆▆▇▆▇▇▇▇▇▇▇█▇▇▇▇█▇█▇▇██▇▇███
train_macro_f1_score,▁▁▂▁▁▂▁▂▂▂▂▃▄▃▄▅▅▅▅▆▆▆▆▆▆▇▆▇▇▇██▇▇▇▇████
avg_dev_fscore,60.10014
avg_test_fscore,65.06258
avg_train_accuracy,93.26047


### EMPTY GPU MEMORY CACHE, AS THE MODEL IS ALREADY SAVED AS PER model_index

In [ ]:
def memory_stats():
    print(torch.cuda.memory_allocated()/1024**2)
    print(torch.cuda.memory_cached()/1024**2)


def allocate():
    x = torch.randn(1024*1024, device='cuda')
    memory_stats()

memory_stats()
torch.cuda.empty_cache()
memory_stats()

2207.974609375
7200.0
2207.974609375
2604.0


## DOING THE TRAINING AGAIN WITH TRANSFER LEARNING USING THE MODEL THAT IS TRAINED ON RAVDESS AND TESS DATASET.

In [10]:
def _supplement_train_data_with_spectrograms(uttm:str, label:int):
    spectrogram_image_path_for_utterance = ""
    DATA_ROOT_PATH='/content/gdrive/MyDrive/emotion_detection_using_sound'
    IEMOCAP_ROOT_FLDR = f"{DATA_ROOT_PATH}/data/Processed_Data_Iemocap"
    IEMOCAP_PROCESSED_FLDR = f"{IEMOCAP_ROOT_FLDR}/iemocap_spectrograms"
    IEMOCAP_TRANSCRIPT_DOC = f"{IEMOCAP_ROOT_FLDR}/iemocapTrans.csv"

    IEMOCAP_EMOTION_FOLDER_MAPPING = {
        "exc":"excited",
        "ang":"angry",
        "fru":"frustrated",
        "hap":"happy",
        "neu":"neutral",
        "sad":"sad"
        }

    # ['angry', 'excited', 'frustrated', 'happy', 'neutral', 'sad']
    LABEL_EMOTION_MAPPING = {
        0:'hap',
        1:'sad',
        2:'neu',
        3:'ang',
        4:'exc',
        5:'fru'
        }
    # print(f"DIALOG: {uttm} | LABEL: {label}")
    # Generate the path of the SPECTROGRAM image
    with open(IEMOCAP_TRANSCRIPT_DOC, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        # print(f"UTTM: {uttm}")
        for index, row in enumerate(reader):
            # print(f"CSV INDEX: {index} {row['to_translate']} | DIALOG: {uttm}")
            csv_string = (row['to_translate']).strip()
            training_data_string = uttm.strip()
            if (row['to_translate']).strip() == uttm.strip() and row['emotion'] == LABEL_EMOTION_MAPPING[label.item()]:
                # print(f"CSV VALUES: {row['to_translate']} | DIALOG: {uttm}")
                utt_emotion = LABEL_EMOTION_MAPPING[label.item()]
                spectrogram_image_folder = IEMOCAP_EMOTION_FOLDER_MAPPING[utt_emotion]
                spectrogram_image_path_for_utterance = f"{IEMOCAP_PROCESSED_FLDR}/{spectrogram_image_folder}/{row['title']}.wav.jpeg"
                break
    if spectrogram_image_path_for_utterance == "" or spectrogram_image_path_for_utterance is None:
        print(f"NOT FOUND UTTERANCE: {uttm} for LABEL: {label.item()} and EMOTION: {LABEL_EMOTION_MAPPING[label.item()]}")
        # sys.exit(0)
    return spectrogram_image_path_for_utterance

def train_with_spectrogram(dataset_name, model, loss_func, trainloader, devloader, testloader,
          n_epochs, optimizer, scheduler, training_step, model_path, log_path,
          metric_path, use_gpu, window, mode):

    model = model.to('cuda:0')
    model.train()

    f = open(log_path, 'a+', encoding='utf-8')

    train_loss_list = []
    train_f1_list = []
    dev_f1_list = []
    test_f1_list = []

    best_fscore = 0
    best_mf1 = 0
    best_accuracy = 0
    best_report = None

    best_test_fscore = 0
    best_test_mf1 = 0
    best_test_accuracy = 0
    best_test_report = None

    step = 0
    early_stopping_step = 0

    #### TEST CODE FOR FINDING ALL THE INCONSISTENCIES
    # from fastai.vision.all import PILImage

    # for epoch in range(n_epochs):
    #     for data in trainloader:
    #         textf, wrdm, label, uttm, spkm, edge_index, edge_attr, _, _, conv = data
    #         spectrogram_image_path = []
    #         if dataset_name == 'IEMOCAP':
    #             zipped_conv_label = zip(conv, label)
    #             for each_conv, conv_label in zipped_conv_label:
    #                 spectrogram_image_path.append(_supplement_train_data_with_spectrograms(each_conv,conv_label))
    #         for each_image_path in spectrogram_image_path:
    #             try:
    #                 image = PILImage.create(each_image_path)
    #             except Exception as e:
    #                 print(f"IMAGE PATH {each_image_path} not found")
    #     for data in devloader:
    #         textf, wrdm, label, uttm, spkm, edge_index, edge_attr, _, _, conv = data
    #         spectrogram_image_path = []
    #         if dataset_name == 'IEMOCAP':
    #             zipped_conv_label = zip(conv, label)
    #             for each_conv, conv_label in zipped_conv_label:
    #                 spectrogram_image_path.append(_supplement_train_data_with_spectrograms(each_conv,conv_label))
    #         for each_image_path in spectrogram_image_path:
    #             try:
    #                 image = PILImage.create(each_image_path)
    #             except Exception as e:
    #                 print(f"IMAGE PATH {each_image_path} not found")
    #     for data in testloader:
    #         textf, wrdm, label, uttm, spkm, edge_index, edge_attr, _, _, conv = data
    #         spectrogram_image_path = []
    #         if dataset_name == 'IEMOCAP':
    #             zipped_conv_label = zip(conv, label)
    #             for each_conv, conv_label in zipped_conv_label:
    #                 spectrogram_image_path.append(_supplement_train_data_with_spectrograms(each_conv,conv_label))
    #         for each_image_path in spectrogram_image_path:
    #             try:
    #                 image = PILImage.create(each_image_path)
    #             except Exception as e:
    #                 print(f"IMAGE PATH {each_image_path} not found")

    # sys.exit(0)
    ####
    for epoch in range(n_epochs):
        losses = []
        preds = []
        labels = []
        masks = []
        num_utt = 0
        print('Epoch : {}'.format(epoch + 1))
        if step > training_step:
            break
        start_time = time.time()
        for data in trainloader:
            # (clen, slen), (clen)
            # Each data that is coming our way - But the entire data is encoded here.
            textf, wrdm, label, uttm, spkm, edge_index, edge_attr, _, _, conv = data
            # print(f"CONVERSATION: {conv} | LABEL: {label}")
            spectrogram_image_path = []
            if dataset_name == 'IEMOCAP':
                zipped_conv_label = zip(conv, label)
                for each_conv, conv_label in zipped_conv_label:
                    spectrogram_image_path.append(_supplement_train_data_with_spectrograms(each_conv,conv_label))
                if use_gpu:
                    textf = textf.cuda()
                    wrdm = wrdm.cuda()
                    uttm = uttm.cuda()
                    spkm = spkm.cuda()
                    edge_index = edge_index.cuda()
                    edge_attr = edge_attr.cuda()
                logits = model(textf, wrdm, uttm, spkm, window, mode, edge_index, edge_attr, spectrogram_image_path, residual=False)
            else:
                conv_len = [int(torch.sum(um).item()) for um in uttm]  # torch.sum(uttm, dim=1).numpy().tolist()
                if dataset_name == 'DailyDialog':
                    logits = model(textf, wrdm, conv_len, uttm, spkm, window, mode, edge_index, edge_attr, use_gpu, residual=False)
                else:
                    logits = model(textf, wrdm, conv_len, edge_index, edge_attr, use_gpu)
                label = torch.cat(label, dim=0)
            if use_gpu:
                label = label.cuda()
            # loss = loss_func(torch.log_softmax(logits, dim=-1), label)
            loss = loss_func(logits, label)

            model.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()

            pred_ = torch.argmax(torch.softmax(logits, dim=-1), dim=1)
            preds.append(pred_.cpu().numpy())
            labels.append(label.data.cpu().numpy())
            losses.append(loss.item() * label.size(0))
            num_utt += label.size(0)
            step += 1
            if step > training_step:
                break
        preds = np.concatenate(preds)
        labels = np.concatenate(labels)

        avg_loss = np.round(np.sum(losses) / num_utt, 4)
        avg_accuracy = round(accuracy_score(labels, preds) * 100, 5)
        if dataset_name == 'DailyDialog':
            avg_fscore = round(f1_score(labels, preds, labels=[0, 2, 3, 4, 5, 6], average='micro') * 100, 5)
        else:
            avg_fscore = round(f1_score(labels, preds, average='weighted') * 100, 5)
        train_mf1 = round(f1_score(labels, preds, average='macro') * 100, 5)
        dev_accuracy, dev_fscore, dev_mf, dev_reports = evaluate_with_spectrogram(dataset_name, model, devloader, use_gpu, window=window, mode=mode)
        test_accuracy, test_fscore, test_mf, test_reports = evaluate_with_spectrogram(dataset_name, model, testloader, use_gpu, window=window, mode=mode)

        print(f"DEV-ACCURACY: {dev_accuracy} || BEST=ACCURACY: {best_accuracy}")
        # Log the Avg. Train Accuracy
        wandb.log({'avg_train_accuracy':avg_accuracy})
        # Log the Train Macro F1 Score
        wandb.log({'train_macro_f1_score':train_mf1})
        # Log the Dev Accuracy
        wandb.log({'dev_accuracy':dev_accuracy})
        # Log the Test Accuracy
        wandb.log({'test_accuracy':test_accuracy})

        if dev_accuracy > best_accuracy:
        # if dev_fscore > best_fscore:
            best_fscore = dev_fscore
            best_mf1 = dev_mf
            best_accuracy = dev_accuracy
            best_report = dev_reports

            best_test_fscore = test_fscore
            best_test_mf1 = test_mf
            best_test_accuracy = test_accuracy
            best_test_report = test_reports

            early_stopping_step = 0
            print("Best Accuracy Till thie Epoch:", best_accuracy)
            # Saving the model
            torch.save(model.state_dict(), model_path)
        else:
            early_stopping_step += 1

        print(f"CURRENT VALUE OF STOPPING: {early_stopping_step}")
        train_loss_list.append(avg_loss)
        # Log the Avg. train loss
        wandb.log({'avg_train_loss':avg_loss})
        train_f1_list.append(avg_fscore)
        # Log the Avg. train f1 score
        wandb.log({'avg_train_fscore':avg_fscore})
        dev_f1_list.append(dev_fscore)
        # Log the Avg. dev f1 score
        wandb.log({'avg_dev_fscore':dev_fscore})
        test_f1_list.append(test_fscore)
        # Log the Avg. test f1 score
        wandb.log({'avg_test_fscore':test_fscore})

        log = 'Train: Epoch {} Loss {}, ACC {}, F1 {}, mF {}'.format(epoch + 1, avg_loss,
                                                                     avg_accuracy, avg_fscore, train_mf1)
        print(log)
        f.write(log + '\n')
        log = 'Validation: ACC {}, F1 {}, mF {}'.format(dev_accuracy, dev_fscore, dev_mf)
        print(log)
        f.write(log + '\n')
        log = 'Test: ACC {}, F1 {}, mF {}'.format(test_accuracy, test_fscore, test_mf)
        print(log)
        f.write(log + '\n')
        print('Epoch {} finished. Elapse {}'.format(epoch + 1, round(time.time() - start_time, 4)))
        if early_stopping_step == 50:
            break
    print('----------------------------------------------')
    f.write('----------------------------------------------')
    log = '\n\n[DEV] best ACC {}, F1 {}, mF {}'.format(best_accuracy, best_fscore, best_mf1)
    f.write(log + '\n')
    print(log)
    f.write(best_report)
    log = '[TEST] best ACC {}, F1 {}, mF {}'.format(best_test_accuracy, best_test_fscore, best_test_mf1)
    f.write(log + '\n')
    print(log)
    f.write(best_test_report)
    f.write('----------------------------------------------\n')
    f.close()
    dump_data = [train_loss_list, train_f1_list, dev_f1_list, test_f1_list]
    pickle.dump(dump_data, open(metric_path, 'wb'))


def evaluate_with_spectrogram(dataset_name, model, dataloader, use_gpu, window, mode):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for data in dataloader:
            textf, wrdm, label, uttm, spkm, edge_index, edge_attr, _, _, conv  = data
            # if dataset_name == 'IEMOCAP':
            spectrogram_image_path = []
            if dataset_name == 'IEMOCAP':
                if spectrogram_mode:
                    zipped_conv_label = zip(conv, label)
                    for each_conv, each_label in zipped_conv_label:
                        spectrogram_image_path.append(_supplement_train_data_with_spectrograms(each_conv,each_label))

                if use_gpu:
                    textf = textf.cuda()
                    wrdm = wrdm.cuda()
                    uttm = uttm.cuda()
                    spkm = spkm.cuda()
                    edge_index = edge_index.cuda()
                    edge_attr = edge_attr.cuda()
                logits = model(textf, wrdm, uttm, spkm, window, mode, edge_index, edge_attr, spectrogram_image_path, residual=False)
            else:
                conv_len = [int(torch.sum(um).item()) for um in uttm]  # torch.sum(uttm, dim=1).numpy().tolist()
                if dataset_name == 'DailyDialog':
                    logits = model(textf, wrdm, conv_len, uttm, spkm, window, mode, edge_index, edge_attr, use_gpu, residual=False)
                else:
                    logits = model(textf, wrdm, conv_len, edge_index, edge_attr, use_gpu)
                # logits = model(textf, wrdm, conv_len, edge_index, edge_attr, use_gpu)
                label = torch.cat(label, dim=0)
            if use_gpu:
                label = label.cuda()
            pred_ = torch.argmax(torch.softmax(logits, dim=1), dim=1)
            preds.append(pred_.cpu().numpy())
            # labels.append(label.data.numpy())
            labels.append(label.data.cpu().numpy())


    preds = np.concatenate(preds)
    labels = np.concatenate(labels)

    avg_accuracy = round(accuracy_score(labels, preds) * 100, 5)

    if dataset_name == 'DailyDialog':
        avg_fscore = round(f1_score(labels, preds, labels=[0, 2, 3, 4, 5, 6], average='micro') * 100, 5)
        report_classes = classification_report(labels, preds, labels=[0, 2, 3, 4, 5, 6], digits=4)
    else:
        avg_fscore = round(f1_score(labels, preds, average='weighted') * 100, 5)
        report_classes = classification_report(labels, preds, digits=4)
    mf1 = round(f1_score(labels, preds, average='macro') * 100, 5)

    model.train()

    return avg_accuracy, avg_fscore, mf1, report_classes


In [13]:
beta=True
batch_size=20
bias=False
dataset_name='IEMOCAP'
# dataset_name='DailyDialog'
encoder_mode='maxpooling'
tr_nhead=6
tr_ff_dim=300
tr_dropout=0.1
attn_mask=True
tr_num_layer=6
max_len=120
bidirectional=True
num_block = 3
cn_nhead=6
# Dimension for feed forward network for Graph Transformer
cn_ff_dim=600
cn_dropout=0.1
edge_dim=300
cn_num_layer=5
edge_mapping=True
root_weight=True
# choice='cn'
choice='both'
valid=0.1
warmup_step=1000
schedule='linear'
training_step=10000
seed=7
n_epochs=150
lr=1e-5
# Weight Decay - l2 normalization
# l2=0
l2=0.05
use_gpu=True
window=10
mode='uso'
# Flag for training with spectrogram images
spectrogram_mode=True

if dataset_name == 'IEMOCAP':
    num_class = 6
else:
    num_class = 7

model_index = str(30)
model_dir = dataset_name + '_C/model'
log_dir = dataset_name + '_C/logs/log'
metric_dir = dataset_name + '_C/logs/metric'

model_path = model_dir + model_index + '.pth'
log_path = log_dir + model_index + '.txt'
metric_path = metric_dir + model_index + '.pkl'

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if dataset_name == 'IEMOCAP':
  pretrain='roberta-base'
  # Dimension for utterance encoder
  sent_dim=300
  hip=10
  print(f"The model getting trained is=> DATASET: {dataset_name}, pretrain: {pretrain}")
  # Using different Mental Model detection class: MentalModelWithSpectrogram
  model_with_spectrogram = MentalModelWithSpectrogram(encoder_type=pretrain, encoder_mode=encoder_mode, sent_dim=sent_dim, tr_nhead=tr_nhead,
                      tr_ff_dim=tr_ff_dim, tr_dropout=tr_dropout, attn_mask=attn_mask, tr_num_layer=tr_num_layer,
                      max_len=max_len, num_class=num_class, bidirectional=bidirectional, num_block=num_block,
                      cn_nhead=cn_nhead, cn_ff_dim=cn_ff_dim, cn_dropout=cn_dropout, edge_dim=edge_dim, bias=bias,
                      cn_num_layer=cn_num_layer, edge_mapping=edge_mapping, beta=beta, root_weight=root_weight, choice=choice)

elif dataset_name == 'DailyDialog':
  pretrain='roberta-large'
  sent_dim=300
  hip=2
  print(f"The model getting trained is=> DATASET: {dataset_name}, pretrain: {pretrain}")
  model = BatchMentalModel(pretrain, encoder_mode, sent_dim, tr_nhead,
                      tr_ff_dim, tr_dropout, attn_mask, tr_num_layer,
                      max_len, num_class, bidirectional, num_block,
                      cn_nhead, cn_ff_dim, cn_dropout, edge_dim, bias,
                      cn_num_layer, edge_mapping, beta, root_weight, choice)
else:
  print(f"The model getting trained is=> DATASET: {dataset_name}, pretrain: {pretrain}")
  model = BatchMentalModelResidual(args.pretrain, args.encoder_mode, args.sent_dim,
                                    args.cn_ff_dim, args.cn_nhead, args.cn_dropout,
                                    args.edge_dim, num_class, bias, args.cn_num_layer,
                                    edge_mapping, beta, root_weight, args.residual_type)

The model getting trained is=> DATASET: IEMOCAP, pretrain: roberta-base


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
import torch.optim as optim

train_loader, dev_loader, test_loader = get_loaders(dataset_name, hip, batch_size, pretrain, valid, True)
loss_func = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(model_with_spectrogram.parameters(), lr=lr, weight_decay=l2)

if schedule == 'linear':
    scheduler = get_linear_schedule_with_warmup(optimizer, warmup_step, training_step)
elif schedule == 'warmup':
    scheduler = get_constant_schedule_with_warmup(optimizer, warmup_step)
else:
    scheduler = get_constant_schedule(optimizer)

#### RETRAINING WITH SPECTROGRAM

In [15]:
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="SKAIG",
    name="SKAIG-With-Spectrogram-With-Batch-Size-20-FastAI-batch-size-20-l2-0.5",
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.01,
    "architecture": "ROBERTA And Graph Neural",
    "dataset": "IEMOCAP",
    "epochs": 150,
    }
)

In [16]:
# IEMOCAP
train_with_spectrogram(dataset_name, model_with_spectrogram, loss_func, train_loader, dev_loader, test_loader, n_epochs, optimizer, scheduler,
            training_step, model_path, log_path, metric_path, use_gpu, window=window, mode=mode)

Epoch : 1
DEV-ACCURACY: 27.93296 || BEST=ACCURACY: 0
Best Accuracy Till thie Epoch: 27.93296
CURRENT VALUE OF STOPPING: 0
Train: Epoch 1 Loss 6.2842, ACC 20.0, F1 21.07561, mF 16.47523
Validation: ACC 27.93296, F1 18.33238, mF 12.29035
Test: ACC 24.95379, F1 16.48024, mF 13.26261
Epoch 1 finished. Elapse 162.6464
Epoch : 2
DEV-ACCURACY: 21.78771 || BEST=ACCURACY: 27.93296
CURRENT VALUE OF STOPPING: 1
Train: Epoch 2 Loss 6.2628, ACC 19.47115, F1 19.54828, mF 16.69504
Validation: ACC 21.78771, F1 18.93124, mF 14.79965
Test: ACC 21.38016, F1 18.40257, mF 16.08974
Epoch 2 finished. Elapse 176.7882
Epoch : 3
DEV-ACCURACY: 19.36685 || BEST=ACCURACY: 27.93296
CURRENT VALUE OF STOPPING: 2
Train: Epoch 3 Loss 6.0949, ACC 19.2053, F1 19.74275, mF 16.69121
Validation: ACC 19.36685, F1 18.43713, mF 15.69266
Test: ACC 19.71657, F1 18.32209, mF 16.66041
Epoch 3 finished. Elapse 156.8611
Epoch : 4
DEV-ACCURACY: 18.62197 || BEST=ACCURACY: 27.93296
CURRENT VALUE OF STOPPING: 3
Train: Epoch 4 Loss 6.426

In [17]:
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

avg_dev_fscore,▂▃▂▂▁▁▁▁▂▂▂▂▃▃▂▂▂▂▃▃▄▅▅▅▅▅▅▅▆▆▇▇▇█▆▄█▆▇▇
avg_test_fscore,▁▃▃▂▁▁▁▁▂▂▁▂▂▂▂▂▃▂▂▃▄▄▆▅▆▅▆▆▆▇▇█▇▆▆▆▇▇▇▇
avg_train_accuracy,▃▃▃▁▂▄▁▃▃▂▂▁▄▃▃▅▁▃▃▄▄▅▁▅▆▄▅▄█▆▇▆▅▃█▃▇▇▆▇
avg_train_fscore,▄▃▃▂▂▄▂▂▄▂▂▁▄▂▃▅▁▃▃▅▅▅▂▅▆▄▄▅█▅▆▆▅▃█▃▇▆▆█
avg_train_loss,██▇█▇▇██▇▇▇▆▇▇▇▆▆▆▅▅▅▄▅▄▄▃▃▃▂▃▃▂▂▃▂▂▁▁▂▁
dev_accuracy,█▄▃▂▁▁▁▁▁▂▂▂▃▃▂▂▂▂▃▃▄▅▄▄▅▅▅▅▆▅▆▆▇▇▅▄▇▆▆▆
test_accuracy,█▅▃▂▁▁▁▁▂▂▁▁▂▂▂▂▃▂▃▃▄▅▅▅▆▅▆▇▇█▇█▇▆▅▄▇▇▇▆
train_macro_f1_score,▃▃▃▁▃▄▁▃▂▂▃▁▂▃▅▅▂▂▃▁▃▅▂▆▆▃▆▄▅▆█▅▅▅█▄██▆▆
avg_dev_fscore,24.49887
avg_test_fscore,21.74646
avg_train_accuracy,25.77488


## WE WILL BE DOING THE PREDICTION PART LATER. BEFORE THAT WE NEED TO INCREASE THE TRAINING ACCURACY USING THE SPECTROGRAM IMAGES(TAKING VOICE INTENSITY INTO ACCOUNT)

### TRYING TO DO PREDICTION USING THE SAVED MODEL

In [ ]:
model_path = "IEMOCAP_C/model_without_spectrogram.pth"
# model_path = "IEMOCAP_C/model_spectrogram_last_2_layer_learning.pth"
print(model_path)

IEMOCAP_C/model_without_spectrogram.pth


In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

'''
label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
'''

# Load the model using torch.load
model_state_dict = torch.load(model_path)

pretrain='roberta-base'
model = AutoModelForSequenceClassification.from_pretrained(pretrain, state_dict=model_state_dict)
tokenizer = AutoTokenizer.from_pretrained(pretrain)

input_text = "Look at this, all of the hairs are standing up on my arm."
tokens = tokenizer(input_text, return_tensors='pt')

with torch.no_grad():
    outputs = model(**tokens)

probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
predicted_class = torch.argmax(probabilities).item()

print("Predicted Emotion:", predicted_class)


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.

Predicted Emotion: 1


In [ ]:
input_text = "This is totally totally Horrible!"
tokens = tokenizer(input_text, return_tensors='pt')

with torch.no_grad():
    outputs = model(**tokens)

probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
predicted_class = torch.argmax(probabilities).item()

print("Predicted Emotion:", predicted_class)

Predicted Emotion: 0


In [ ]:
input_text = "What the hell do you think of yourself?"
tokens = tokenizer(input_text, return_tensors='pt')

with torch.no_grad():
    outputs = model(**tokens)

probabilities = torch.nn.functional.softmax(outputs.logits, dim=1)
predicted_class = torch.argmax(probabilities).item()

print("Predicted Emotion:", predicted_class)

Predicted Emotion: 0


In [ ]:
# def predict_emotion(input_text, pretrained_model, model_path):
#     pretrained_model.load_state_dict(torch.load(model_path))
#     pretrained_model.to('cuda:0')
#     pretrained_model.eval()

#     # tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
#     # # Tokenize the input
#     # inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
#     # print(inputs)
#     # # Make predictions
#     with torch.no_grad():
#         predictions = pretrained_model(input_text)


#     # Interpret predictions
#     probabilities = F.softmax(predictions, dim=1)
#     predicted_class = torch.argmax(probabilities, dim=1).item()

#     print(f"Predicted class: {predicted_class}")


def preprocess_speaker(speaker_info):
    # Your preprocessing logic for speaker information
    # ...

    # Dummy example: Assuming one-hot encoding for demonstration
    speaker_classes = ["M", "F"]
    one_hot_encoded = [int(speaker_info == speaker) for speaker in speaker_classes]

    return torch.tensor(one_hot_encoded)

def predict_emotion(input_text, speaker_information, dataset_name, model_path):
    inputs_tokenized = []
    if dataset_name == 'IEMOCAP':
        pretrain='roberta-base'
        tokenizer = AutoTokenizer.from_pretrained(pretrain)
        # Tokenize the input
        inputs_tokenized = tokenizer.encode_plus(input_text, add_special_tokens=True, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
        print(type(inputs_tokenized), type(inputs_tokenized['input_ids']))
        # Adjust tokens based on your model's specific requirements
        input_ids = inputs_tokenized['input_ids'].to('cuda:0')
        attention_mask = inputs_tokenized['attention_mask'].to('cuda:0')
        speaker_input = preprocess_speaker(speaker_information)
        # model = MentalModel(encoder_type=pretrain)
        # model = AutoModelForSequenceClassification.from_pretrained(pretrain)
        model = MentalModel(encoder_type=pretrain, encoder_mode=encoder_mode, sent_dim=sent_dim, tr_nhead=tr_nhead,
                        tr_ff_dim=tr_ff_dim, tr_dropout=tr_dropout, attn_mask=attn_mask, tr_num_layer=tr_num_layer,
                        max_len=max_len, num_class=num_class, bidirectional=bidirectional, num_block=num_block,
                        cn_nhead=cn_nhead, cn_ff_dim=cn_ff_dim, cn_dropout=cn_dropout, edge_dim=edge_dim, bias=bias,
                        cn_num_layer=cn_num_layer, edge_mapping=edge_mapping, beta=beta, root_weight=root_weight, choice=choice)

    elif dataset_name == 'DailyDialog':
        pretrain='roberta-large'
        tokenizer = AutoTokenizer.from_pretrained(pretrain)
        # Tokenize the input
        inputs_tokenized = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
        model = BatchMentalModel()
    else:
        model = BatchMentalModelResidual()

    # print(f"Loading the Model: {model} from the path:{model_path}!")
    model.load_state_dict(torch.load(model_path), strict=False)

    model.eval()
    model.to('cuda:0')
    # print(inputs)
    # # Make predictions
    with torch.no_grad():
        predictions = model(**inputs_tokenized, mask=None)

    print(predictions)

    # Interpret predictions
    probabilities = F.softmax(predictions, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

    labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
    print(f"Predicted class: {predicted_class}")

    predicted_label = label_names[predicted_class]
    print(predicted_label)


input_text="This is an excellect piece of information! Well Done."
speaker_information="M"
print(model_path, dataset_name)

predict_emotion(input_text, speaker_information, dataset_name, model_path)

IEMOCAP_C/model_without_spectrogram.pth IEMOCAP
<class 'transformers.tokenization_utils_base.BatchEncoding'> <class 'torch.Tensor'>


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: MentalModel.forward() got an unexpected keyword argument 'input_ids'

#### TRYING TO CREATE KNOWLEDGE GRAPHS FOR SENTENCES

In [ ]:
!pip install graph-transformer

  Preparing metadata (setup.py) ... done
  Created wheel for graph-transformer: filename=graph_transformer-0.1-py3-none-any.whl size=1388 sha256=0777bcd37e6a88864922a3bb706fbc9c462e079fad7bfeae139b34cfd1dda3ae
  Stored in directory: /root/.cache/pip/wheels/9f/ac/14/356ad0100b08277a93aaa48afd2ee90fc82297740daa89fd08
Successfully built graph-transformer


In [ ]:
!pip install graph-transformer-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.1/866.1 kB 25.4 MB/s eta 0:00:00


In [ ]:
import torch
from graph-transformer import GraphTransformerModel

model = GraphTransformerModel(
        node_dim = 512,
        edge_dim = 512,
        num_blocks = 3, # number of graph transformer blocks
        num_heads = 8,
        last_average=True, # whether to average or concatenation at the last block
        model_dim=None # if None, node_dim will be used as the dimension of the graph transformer block
)

nodes = torch.randn(1, 128, 512)
edges = torch.randn(1, 128, 128, 512)
adjacency = torch.ones(1, 128, 128)

nodes = model(nodes, edges, adjacency)

SyntaxError: invalid syntax (<ipython-input-20-b94fd3780d16>, line 2)

In [ ]:
import pickle
import pandas as pd

# with open('bert_data/IEMOCAP/IEMOCAP_graph_hip7_new.pkl', 'rb') as f:
#     data = pickle.load(f)

pickled_df = pd.DataFrame(pd.read_pickle("bert_data/IEMOCAP/IEMOCAP_graph_hip7_new.pkl"))
pickled_df

,train,test
0,"[[Hey, Isabella., Hey, how's it going, Joe?, I'm all right., Uh-, So I got some good news., Yep., You're not pregnant or anything., I know you've been trying to have a baby, right?, I have some better news than that., Yes, What would you want more than anything in this world?, How about maybe for me to possibly stay in L.A.?, Which one?, I got--U.S.C.. I got- Yeah, I got the acceptance letter., Yes, me, I got in. Thank you very much., Good for you., Yeah, I know, my mom called me. I was with my girlfriend in Boston, and uh- she got the letter and she called me, she said I don't know, do yo...","[[Why does that bother you?, Well, maybe maybe he just wanted to see her again., Nobody comes seven hundred miles just to see., What are you talking about? He grew up next to the girl his whole life, why wouldn't he want to see her again? Don't look at me like that. He hasn't told me anything he didn't told you., Why do you think he's even thinking that?, He's got that about it., Well, so what?, What's going on here, Joe?, Now listen., How do you know why she's waited?, Look, it's a beautiful day outside, why are we arguing?, Well, what do you want me to do about it? What do you want?,..."
1,"[[2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 4, 4, 4, 4, 4, 4, 2, 0, 4, 0, 4, 4, 4, 2, 0, 4, 0, 0, 4, 4, 0, 2, 4, 4, 4, 2, 0, 0, 2, 4, 4, 4, 4, 4, 4, 4, 4, 2, 0, 0, 4, 2, 0, 0, 0, 0], [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0, 0, 4, 0, 4, 4, 0, 0, 4, 4, 0, 4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 0, 0, 0, 4, 4, 0], [5, 5, 2, 5, 5, 2, 2, 2, 5, 2, 2, 5, 5, 5, 2, 5, 2, 5, 2, 5, 5, 2, 5, 2, 5, 5, 2, 5, 2, 5, 2, 5, 5, 2, 2, 2, 2, 2, 2], [0, 2, 4, 5, 5, 5, 5, 5, 4, 2, 0, 3, 5, 3, 5, 5, 2, 5, 5, 5, 3, 5, 4, 3, 5, 4, 3, 5, 5, 5, 4, 5, 3, 3, 5, 5, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 3, 3], [2, 4, 4, 4, 4, 4, 4...","[[2, 2, 2, 5, 2, 2, 5, 3, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3], [2, 1, 1, 1, 1, 3, 3, 5, 1, 5, 1, 1, 3, 1, 1, 5, 3, 5, 3, 5, 5, 5, 5, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 2, 5, 2, 5, 2, 5, 2, 2, 5, 2, 5, 2, 5, 2, 5, 2, 5, 2, 2], [2, 2, 2, 2, 2, 5, 5, 3, 2, 5, 5, 5, 5, 2, 5, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 3, 5], [2, 2, 2, 2, 2, 2, 5, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 2, 5, 2, 2, 5, 5, 2, 5, 5, 5, 2, 5, 2, 2, 2, 5, 2, 3, 2, 5, 5, 2, 2, 3..."
2,"[[M, F, M, F, F, M, M, F, F, F, M, F, F, M, F, F, M, F, M, F, M, F, M, F, M, F, F, F, M, F, M, M, F, M, F, M, F, F, M, F, F, M, F, F, M, M, M, F, F, M, M, M, F, M, F, M], [M, F, M, F, F, F, M, F, F, F, M, M, F, M, M, F, M, F, F, M, M, M, F, M, F, M, F, M, F, M, F, M, F, M, F, M, F, M, M, F, F, M, F, M], [M, M, F, M, M, F, F, M, M, F, F, M, M, M, F, M, F, M, F, M, M, F, M, F, M, M, F, M, F, M, F, M, M, M, F, M, F, M, F], [M, F, M, M, M, F, M, M, M, M, M, F, M, F, M, F, M, F, M, F, F, M, M, F, M, M, F, M, F, M, M, M, F, F, M, M, F, M, M, M, M, M, M, M, M, M, M, F, F, F], [M, M, F, M, F, M, F...","[[M, M, F, M, M, F, M, F, M, M, M, M, F, M, F, M, M, F, M, F, M, F, M], [M, F, F, F, F, M, M, M, F, M, F, F, M, F, F, F, M, F, M, M, F, F, F, F, M, M, F, M, F, F, F, M, F, F, M, F, M, M, M, F, M, F, M, F, F, F, F, M, F], [M, F, M, F, M, F, M, F, M, F, M, F, M, F, M, F, M, F, M, F, F, M, F, F, M, F, M, F, M, F, F, M, F, M, F, M, F, M, F, M, F, M], [F, F, F, F, F, M, F, M, F, M, F, M, F, F, F, M, F, F, M, F, M, F, F, M, F, M, F, F, M, F, M, F, M, F, M, M, F, M, F, M], [F, M, F, M, F, M, F, M, F, F, M, M, F, M, M, F, M, F, M, F, M, F, M, F, M, F, M, F, M, F, M, M, M, F, M, F, M, M, M, M, M, F..."
3,"{'edge_index': [[[ 0 0 0 ... 55 55 55], [ 0 1 2 ... 51 53 55]], [[ 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 